In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Resize all images to 224 * 224 pixels
IMAGE_SIZE = [224, 224]

In [4]:
train_data_path = 'xray_dataset/train'
test_data_path = 'xray_dataset/test'
val_data_path = 'xray_dataset/val'

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 9s 0us/step


In [5]:
# Do not train existing weights, they already derive from 'imagenet'
for layer in vgg.layers:
    layer.trainable = False

In [23]:
# For getting number of classes, aka Normal and Pneumonia
folders = glob('xray_dataset/train/*')
folders

['xray_dataset/train/NORMAL', 'xray_dataset/train/PNEUMONIA']

In [34]:
# Layers
x = Flatten()(vgg.output)

In [35]:
prediction = Dense(len(folders), activation='softmax')(x)
prediction

<tf.Tensor 'dense_1/Softmax:0' shape=(None, 2) dtype=float32>

In [36]:
model = Model(inputs=vgg.input, outputs=prediction)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   